## Import ##

In [1]:
###
is_on_kaggle = True
# is_on_kaggle = False
###
is_debugging = True
# is_debugging = False

In [2]:
# import packages
import os
import sys
import multiprocessing
from pathlib import Path
import random
from collections import defaultdict
from glob import glob
import pickle
from joblib import Parallel, delayed
import gc
from tqdm.notebook import tqdm
from tabulate import tabulate
import yaml
import datetime
from logging import getLogger
import wandb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GroupKFold, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score

import cv2
import pydicom
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam, AdamW
from torchvision import models
import torchvision.transforms.v2 as t
from torchvision.transforms.v2 import (Resize, Compose, RandomHorizontalFlip, 
                                       ColorJitter, RandomAffine, RandomErasing, ToTensor)
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
import timm
import albumentations as A



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning

## Constants ##

In [5]:
# config file
cfg = {
    'general': {
        'project_name': '202309-rsna-atd',
        'base_path': '../../data', 
        'exp_name': 'exp008',
        'seed': 42,
        'use_wandb': True,
    },
    'data': {
        'n_folds': 5,
        'fold_i': [0], 
        'batch_size': 16,
        'batch_size_inference': 'not used',  # 16,
        'kls_slice_start': 0.6, 
        'b_e_slice_start': 'not used',  # 0.0
        'kls_stride': 4, 
        'b_e_stride': 'not used',  # 16, 
        'calc_cv_score': False, 
        'apply_aug': True,
    }, 
    'model': {
        'model_type': 'kls',  # kls, b_e
        'model_name': 'maxvit_tiny_tf_384.in1k',
        'pretrained': True, 
        'in_chans': 1, 
        'num_classes': 0, # to use as backbone
        'global_pool': 'max',
        'drop_rate': 0.5, 
        'drop_path_rate': 0.2, 
        'hidden_dim': 128,
        'p_dropout': 0.3,
        'lr': 1.0e-4, 
    },
    'pl_params': {
        'accelerator': 'auto',
        'max_epochs': 2, 
        'precision': 16, # 16 or 32
        'enable_progress_bar': True, 
        'limit_train_batches': 1.0, 
        'limit_val_batches': 1.0, 
    }
}

if is_debugging:
    cfg['general']['use_wandb'] = False
    cfg['pl_params']['limit_train_batches'] = 0.01
    cfg['pl_params']['limit_val_batches'] = 0.01



In [6]:
# misc functions
torch.cuda.empty_cache()
# multiprocessing.set_start_method('spawn', force=True)
seed_everything(cfg['general']['seed'], workers=True)
num_workers = os.cpu_count()
gpu_count = torch.cuda.device_count()
print('num_workers:', num_workers)
print('gpu_count:', gpu_count)

def random_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.cuda.manual_seed(seed)    

def start_wandb():
    wandb.init(
        project=cfg['general']["project_name"],
        name=cfg['general']['exp_name'],
        config=cfg,
    )
    wandb_logger = WandbLogger(
        project=cfg['general']['project_name'],
        name=cfg['general']['exp_name'],
        config=cfg,
    )
    return wandb_logger


random_seed(cfg['general']['seed'])
logger = start_wandb() if cfg['general']['use_wandb'] else None



## Paths ##

In [7]:
if is_on_kaggle:
    BASE_PATH = '/kaggle/input'
else:
    BASE_PATH = '.'

TRAIN_IMG_PATHS = [
    f'{BASE_PATH}/rsna-abdominal-trauma-detection-png-pt1', 
    f'{BASE_PATH}/rsna-abdominal-trauma-detection-png-pt2',
    f'{BASE_PATH}/rsna-2023-abdominal-trauma-detection-pngs-3-8',
    f'{BASE_PATH}/rsna-abdominal-trauma-detection-png-pt4',
    f'{BASE_PATH}/rsna-abdominal-trauma-detection-png-pt5',
    f'{BASE_PATH}/rsna-abdominal-trauma-detection-png-pt6',
    f'{BASE_PATH}/rsna-abdominal-trauma-detection-pngs-pt7',
    f'{BASE_PATH}/rsna-2023-abdominal-trauma-detection-pngs-18',
]
TRAIN_DF_PATH = f'{BASE_PATH}/rsna-2023-abdominal-trauma-detection/train.csv'
B_E_POS_PATH = f'{BASE_PATH}/b-e-data/b_e_pos.csv'
B_E_NEG_PATH = f'{BASE_PATH}/b-e-data/sample_b_e_neg.csv'

# make directory to save models
if not os.path.exists('models'):
    os.mkdir('models')
if is_on_kaggle:
    MODEL_PATH = f"./models/{cfg['model']['model_name']}_{cfg['general']['exp_name']}_{cfg['model']['model_type']}"
else:
    MODEL_PATH = f"../models/{cfg['model']['model_name']}_{cfg['general']['exp_name']}_{cfg['model']['model_type']}"



## Preparing data ##

In [6]:
class AbdominalKLSData(Dataset):

    def __init__(self, cfg, train_df, train_img_dirs, slice_start, stride=10, apply_aug=True, num_fold=5):
        super().__init__()
        self.cfg = cfg
        self.slice_start = slice_start
        self.train_df = train_df
        self.train_img_paths = self._fetch_and_sample_train_images(train_img_dirs, stride=stride)
        self.augmentation = apply_aug

        self.gkf = GroupKFold(n_splits=num_fold)
        groups = np.array([os.path.basename(img_path).split('_')[0] for img_path in self.train_img_paths])
        self.fold_data = list(self.gkf.split(self.train_img_paths, groups=groups))

        self.normalize = Compose([
            # Resize((256, 256), antialias=True),
            # RandomHorizontalFlip(),  # Randomly flip images left-right
            # ColorJitter(brightness=0.2),  # Randomly adjust brightness
            # ColorJitter(contrast=0.2),  # Randomly adjust contrast
            # RandomAffine(degrees=0, shear=10),  # Apply shear transformation
            # RandomAffine(degrees=0, scale=(0.8, 1.2)),  # Apply zoom transformation
            # RandomErasing(p=0.2, scale=(0.02, 0.2)),  # Coarse dropout
            ToTensor(),
        ])

        # augmentation
        # flip
        self.aug_h_flip = A.HorizontalFlip(p=0.5)
        self.aug_v_flip = A.VerticalFlip(p=0.5)
        # elastic and grid
        self.aug_distortion = A.GridDistortion(p=0.5)
        self.aug_elastic = A.ElasticTransform(p=0.5)
        # affine
        self.aug_affine = A.Affine(
            scale=(0.8, 1.2),
            translate_percent=(0.0, 0.2),
            rotate=(-45, 45),
            shear=(-15, 15),
            p=0.5)
        # self.aug_affine = A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.8)
        # clahe
        self.aug_clahe = A.CLAHE(p=0.5)
        # bright
        self.aug_bright = A.OneOf([
            A.RandomGamma(gamma_limit=(50, 150), p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.5)
        ], p=0.5)
        # cutout
        self.aug_cutout = A.CoarseDropout(max_height=8, max_width=8, p=0.5)
        # randomcrop
        self.aug_randomcrop = A.RandomResizedCrop(
            height=256,
            width=256,
            scale=(0.8, 1.0),
            ratio=(3/4, 4/3),
            p=0.5)

    def __len__(self):
        return len(self.train_img_paths)

    def __getitem__(self, idx):
        sample_img_path = self.train_img_paths[idx]
        patient_id = int(os.path.basename(sample_img_path).split('_')[0])

        # preprocess image
        img = self._process_img(sample_img_path)
        # img.shape: (256, 256)

        # augmentation
        if self.augmentation:
            img = self.aug_h_flip(image=img)["image"]
            img = self.aug_v_flip(image=img)["image"]
            img = self.aug_distortion(image=img)["image"]
            img = self.aug_clahe(image=img)["image"]
            img = self.aug_affine(image=img)["image"]
            img = self.aug_bright(image=img)["image"]
            img = self.aug_cutout(image=img)["image"]
            img = self.aug_randomcrop(image=img)["image"]

        img = img.astype('float32') / 255
        # img.shape: (256, 256)

        img = torch.tensor(img, dtype=torch.float).unsqueeze(dim=0)
        # img.shape: (1, 256, 256)
        if self.cfg['model']['model_name'] == 'maxvit_tiny_tf_384.in1k':
            img = Compose([Resize((384, 384), antialias=True)])(img)
        img = self.normalize(img)
        # img.shape: (1, 256, 256)

        # labels
        label = self.train_df[self.train_df.patient_id == patient_id].values[0][1:-1]
        kidney = np.argmax(label[4:7], keepdims=False)
        liver = np.argmax(label[7:10], keepdims=False)
        spleen = np.argmax(label[10:], keepdims=False)

        return {
            'patient_id': patient_id,
            'image': img,
            'kidney': kidney,
            'liver': liver,
            'spleen': spleen,
        }

    def _fetch_and_sample_train_images(self, img_dirs, stride):
        """
        Fetches and samples at least one training image per series from the training directories.
        """
        print('Fetching and sampling training images...')
        paths = []
        patients_to_series_to_img_paths = defaultdict(lambda: defaultdict(list))
        for img_dir in img_dirs:
            for filename in tqdm(os.listdir(img_dir)):
                patient_id, series_id, _ = filename.split('_')
                patients_to_series_to_img_paths[patient_id][series_id].append(os.path.join(img_dir, filename))

            for patient_id, series_to_img_paths in patients_to_series_to_img_paths.items():
                for series_id, imgs in series_to_img_paths.items():
                    # sort by instance number
                    sorted_img_paths = sorted(imgs, key=lambda x: int(x.split('_')[-1].split('.')[0]))
                    start_index = int(len(sorted_img_paths) * self.slice_start)
                    end_index = int(len(sorted_img_paths) * (self.slice_start + 0.2))
                    roi = sorted_img_paths[start_index:end_index]
                    for img_path in roi[::stride]:
                        paths.append(img_path)

        return paths

    def _process_img(self, img_path):
        image = cv2.imread(img_path)
        # image = image.astype('float32') / 255
        image = (image.astype('float32') * 255).astype('uint8')
        greyscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        greyscale = cv2.resize(greyscale, (256, 256))
        return greyscale

    def get_one_fold(self, fold=0):
        train_indices, val_indices = self.fold_data[fold]
        train_data = Subset(self, train_indices)
        val_data = Subset(self, val_indices)
        return train_data, val_data


class AbdominalBEData(Dataset):

    def __init__(self, cfg, train_df, data_path, b_e_pos, sample_b_e_neg, apply_aug=True, num_fold=5):
        super().__init__()
        self.cfg = cfg
        self.train_df = train_df
        self.b_e_data = pd.concat([b_e_pos, sample_b_e_neg])
        self.train_img_paths = self._fetch_train_image_paths(data_path, self.b_e_data)
        self.augmentation = apply_aug

        self.gkf = GroupKFold(n_splits=num_fold)
        groups = np.array([os.path.basename(img_path).split('_')[0] for img_path in self.train_img_paths])
        self.fold_data = list(self.gkf.split(self.train_img_paths, groups=groups))

        self.normalize = Compose([
            # Resize((256, 256), antialias=True),
            # RandomHorizontalFlip(),  # Randomly flip images left-right
            # ColorJitter(brightness=0.2),  # Randomly adjust brightness
            # ColorJitter(contrast=0.2),  # Randomly adjust contrast
            # RandomAffine(degrees=0, shear=10),  # Apply shear transformation
            # RandomAffine(degrees=0, scale=(0.8, 1.2)),  # Apply zoom transformation
            # RandomErasing(p=0.2, scale=(0.02, 0.2)),  # Coarse dropout
            ToTensor(),
        ])

        # augmentation
        # flip
        self.aug_h_flip = A.HorizontalFlip(p=0.5)
        self.aug_v_flip = A.VerticalFlip(p=0.5)
        # elastic and grid
        self.aug_distortion = A.GridDistortion(p=0.5)
        self.aug_elastic = A.ElasticTransform(p=0.5)
        # affine
        self.aug_affine = A.Affine(
            scale=(0.8, 1.2),
            translate_percent=(0.0, 0.1),
            rotate=(-35, 35),
            shear=(-15, 15),
            p=0.5)
        # self.aug_affine = A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.8)
        # clahe
        self.aug_clahe = A.CLAHE(p=0.5)
        # bright
        self.aug_bright = A.OneOf([
            A.RandomGamma(gamma_limit=(60, 140), p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.5)
        ], p=0.5)
        # cutout
        self.aug_cutout = A.CoarseDropout(max_height=8, max_width=8, p=0.5)
        # randomcrop
        self.aug_randomcrop = A.RandomResizedCrop(
            height=256,
            width=256,
            scale=(0.8, 1.0),
            ratio=(3/4, 4/3),
            p=0.5)

    def __len__(self):
        return len(self.train_img_paths)

    def __getitem__(self, idx):
        sample_img_path = self.train_img_paths[idx]
        patient_id = int(os.path.basename(sample_img_path).split('_')[0])

        # preprocess image
        img = self._process_img(sample_img_path)
        # img.shape: (256, 256)

        # augmentation
        if self.augmentation:
            img = self.aug_h_flip(image=img)["image"]
            img = self.aug_v_flip(image=img)["image"]
            img = self.aug_distortion(image=img)["image"]
            img = self.aug_clahe(image=img)["image"]
            img = self.aug_affine(image=img)["image"]
            img = self.aug_bright(image=img)["image"]
            img = self.aug_cutout(image=img)["image"]
            img = self.aug_randomcrop(image=img)["image"]

        img = img.astype('float32') / 255
        # img.shape: (256, 256)

        img = torch.tensor(img, dtype=torch.float).unsqueeze(dim=0)
        # img.shape: (1, 256, 256)
        if self.cfg['model']['model_name'] == 'maxvit_tiny_tf_384.in1k':
            img = Compose([Resize((384, 384), antialias=True)])(img)
        img = self.normalize(img)
        # img.shape: (1, 256, 256)

        # labels
        bowel = self.b_e_data[self.b_e_data.filename == sample_img_path].bowel.values[0]
        extravasation = self.b_e_data[self.b_e_data.filename == sample_img_path].extravasation.values[0]

        return {
            'patient_id': patient_id,
            'image': img,
            'bowel': bowel,
            'extravasation': extravasation,
        }

    def _fetch_train_image_paths(self, data_path, b_e_data):
        # get paths from both b_e_pos and sample_b_e_neg
        paths = []
        # add base_path to filename
        b_e_data['filename'] = b_e_data['filename'].apply(lambda x: os.path.join(data_path, x))
        paths.extend(b_e_data['filename'])
        # shuffle
        random.shuffle(paths)
        return paths

    def _process_img(self, img_path):
        image = cv2.imread(img_path)
        # image = image.astype('float32') / 255
        image = (image.astype('float32') * 255).astype('uint8')
        greyscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        greyscale = cv2.resize(greyscale, (256, 256))
        return greyscale

    def get_one_fold(self, fold=0):
        train_indices, val_indices = self.fold_data[fold]
        train_data = Subset(self, train_indices)
        val_data = Subset(self, val_indices)
        return train_data, val_data


## Model Architecture ##

In [7]:
# Model Architecure
class KLSNet(pl.LightningModule):

    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.backbone = timm.create_model(
            model_name=cfg['model']['model_name'],
            pretrained=cfg['model']['pretrained'],
            in_chans=cfg['model']['in_chans'],
            num_classes=cfg['model']['num_classes'],
            global_pool=cfg['model']['global_pool'],
            drop_rate=cfg["model"]["drop_rate"],
            drop_path_rate=cfg["model"]["drop_path_rate"],
        )
        # for param in self.backbone.parameters():
        #     param.requires_grad = False

        self.in_features = self.backbone.num_features  # 1280
        hidden_dim = cfg['model']['hidden_dim']
        self.neck = nn.Sequential(
            nn.Linear(self.in_features, hidden_dim),
            nn.ReLU(),
            nn.Dropout(cfg['model']['p_dropout']),
        )

        self.kidney = nn.Linear(hidden_dim, 3)
        self.liver = nn.Linear(hidden_dim, 3)
        self.spleen = nn.Linear(hidden_dim, 3)

        self.cce = nn.CrossEntropyLoss(label_smoothing=0.05, weight=torch.tensor(cfg['model']['kls_weights']))

        self.train_epoch_loss = []
        self.val_epoch_loss = []
        self.probs = defaultdict(list)
        self.targets = defaultdict(list)
        self.auc_scores = dict()

    def forward(self, x):
        # extract features
        x = self.backbone(x)
        x = self.neck(x)

        # output logits
        kidney = self.kidney(x)
        liver = self.liver(x)
        spleen = self.spleen(x)

        return kidney, liver, spleen

    def training_step(self, batch, batch_idx):
        inputs = batch['image']
        kidney = batch['kidney']
        liver = batch['liver']
        spleen = batch['spleen']

        k, l, s = self.forward(inputs)
        k_loss = self.cce(k, kidney)
        l_loss = self.cce(l, liver)
        s_loss = self.cce(s, spleen)
        loss = k_loss + l_loss + s_loss
        self.train_epoch_loss.append(loss.item())

        self.log('train_loss', loss, prog_bar=True, logger=True, on_epoch=True, on_step=True, sync_dist=True)
        return loss

    # def on_train_epoch_end(self):
    #     avg_loss = np.mean(self.train_epoch_loss)
    #     self.log('avg_train_loss', avg_loss, prog_bar=True)
    #     self.train_epoch_loss.clear()

    def validation_step(self, batch, batch_idx):
        inputs = batch['image']
        kidney = batch['kidney']
        liver = batch['liver']
        spleen = batch['spleen']

        k, l, s = self.forward(inputs)
        k_loss = self.cce(k, kidney)
        l_loss = self.cce(l, liver)
        s_loss = self.cce(s, spleen)
        loss = k_loss + l_loss + s_loss
        self.val_epoch_loss.append(loss.item())

        self.probs['k'].extend(F.softmax(k, dim=1).detach().cpu().numpy())
        self.probs['l'].extend(F.softmax(l, dim=1).detach().cpu().numpy())
        self.probs['s'].extend(F.softmax(s, dim=1).detach().cpu().numpy())
        self.targets['k'].extend(kidney.detach().cpu().numpy())
        self.targets['l'].extend(liver.detach().cpu().numpy())
        self.targets['s'].extend(spleen.detach().cpu().numpy())

        self.log('val_loss', loss, prog_bar=True, logger=True, on_epoch=True, on_step=True, sync_dist=True)
        return loss

    def on_validation_epoch_end(self):
        avg_loss = np.mean(self.val_epoch_loss)

        for t in ['k', 'l', 's']:
            self.auc_scores[t] = roc_auc_score(
                self.targets.get(t),
                self.probs.get(t),
                multi_class='ovo', labels=[0, 1, 2])

        # self.log('avg_val_loss', avg_loss, prog_bar=True)
        self.log('val_auc_score_k', self.auc_scores.get('k'), prog_bar=True, sync_dist=True)
        self.log('val_auc_score_l', self.auc_scores.get('l'), prog_bar=True, sync_dist=True)
        self.log('val_auc_score_s', self.auc_scores.get('s'), prog_bar=True, sync_dist=True)
        self.val_epoch_loss.clear()
        self.probs.clear()
        self.targets.clear()
        self.auc_scores.clear()

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=float(self.cfg['model']['lr']))
        # optimizer = AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=float(self.cfg['model']['lr']))
        return optimizer

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        pass


# Model Architecure
class BENet(pl.LightningModule):

    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.backbone = timm.create_model(
            model_name=cfg['model']['model_name'],
            pretrained=cfg['model']['pretrained'],
            in_chans=cfg['model']['in_chans'],
            num_classes=cfg['model']['num_classes'],
            global_pool=cfg['model']['global_pool'],
            drop_rate=cfg["model"]["drop_rate"],
            drop_path_rate=cfg["model"]["drop_path_rate"],
        )
        # for param in self.backbone.parameters():
        #     param.requires_grad = False

        self.in_features = self.backbone.num_features  # 1280
        hidden_dim = cfg['model']['hidden_dim']
        self.neck = nn.Sequential(
            nn.Linear(self.in_features, hidden_dim),
            nn.ReLU(),
            nn.Dropout(cfg['model']['p_dropout']),
        )

        self.bowel = nn.Linear(hidden_dim, 2)
        self.extravasation = nn.Linear(hidden_dim, 2)

        self.cce_b = nn.CrossEntropyLoss(label_smoothing=0.05, weight=torch.tensor(cfg['model']['b_weights']))
        self.cce_e = nn.CrossEntropyLoss(label_smoothing=0.05, weight=torch.tensor(cfg['model']['e_weights']))

        self.train_epoch_loss = []
        self.val_epoch_loss = []
        self.probs = defaultdict(list)
        self.targets = defaultdict(list)
        self.auc_scores = dict()

    def forward(self, x):
        # extract features
        x = self.backbone(x)
        x = self.neck(x)

        # output logits
        bowel = self.bowel(x)
        extravsation = self.extravasation(x)

        return bowel, extravsation

    def training_step(self, batch, batch_idx):
        inputs = batch['image']
        bowel = batch['bowel']
        extravasation = batch['extravasation']

        b, e = self.forward(inputs)
        b_loss = self.cce_b(b, bowel)
        e_loss = self.cce_e(e, extravasation)
        loss = b_loss + e_loss
        self.train_epoch_loss.append(loss.item())

        self.log('train_loss', loss, prog_bar=True, logger=True, on_epoch=True, on_step=True, sync_dist=True)
        return loss

    # def on_train_epoch_end(self):
    #     avg_loss = np.mean(self.train_epoch_loss)
    #     self.log('avg_train_loss', avg_loss, prog_bar=True)
    #     self.train_epoch_loss.clear()

    def validation_step(self, batch, batch_idx):
        inputs = batch['image']
        bowel = batch['bowel']
        extravasation = batch['extravasation']

        b, e = self.forward(inputs)
        b_loss = self.cce_b(b, bowel)
        e_loss = self.cce_e(e, extravasation)
        loss = b_loss + e_loss
        self.val_epoch_loss.append(loss.item())

        self.probs['b'].extend(F.softmax(b, dim=1).detach().cpu().numpy())
        self.probs['e'].extend(F.softmax(e, dim=1).detach().cpu().numpy())
        self.targets['b'].extend(bowel.detach().cpu().numpy())
        self.targets['e'].extend(extravasation.detach().cpu().numpy())

        self.log('val_loss', loss, prog_bar=True, logger=True, on_epoch=True, on_step=True, sync_dist=True)
        return loss

    def on_validation_epoch_end(self):
        avg_loss = np.mean(self.val_epoch_loss)

        for t in ['b', 'e']:
            y_true = np.ravel(self.targets.get(t))
            prob_array = np.array(self.probs.get(t))
            if len(np.unique(y_true)) != 2:
                return -1
            self.auc_scores[t] = roc_auc_score(y_true, prob_array[:, 1])

        # self.log('avg_val_loss', avg_loss, prog_bar=True)
        self.log('val_auc_score_b', self.auc_scores.get('b'), prog_bar=True, sync_dist=True)
        self.log('val_auc_score_e', self.auc_scores.get('e'), prog_bar=True, sync_dist=True)
        self.val_epoch_loss.clear()
        self.probs.clear()
        self.targets.clear()
        self.auc_scores.clear()

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=float(self.cfg['model']['lr']))
        # optimizer = AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=float(self.cfg['model']['lr']))
        return optimizer

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        pass



## Training and Validating ##

In [10]:
# prepare dataset
print("Preparing dataset...")
train_df = pd.read_csv(TRAIN_DF_PATH)
# kidney, liver, spleen dataset
if cfg['model']['model_type'] == 'kls':
    print('n_folds:', cfg['data']['n_folds'])
    print('kls_slice_start:', cfg['data']['kls_slice_start'])
    print('kls_stride:', cfg['data']['kls_stride'])
    print('apply_aug:', cfg['data']['apply_aug'])
    dataset = AbdominalKLSData(
        train_df,
        TRAIN_IMG_PATHS,
        slice_start=cfg['data']['kls_slice_start'],
        stride=cfg['data']['kls_stride'],
        apply_aug=cfg['data']['apply_aug'],
        num_fold=cfg['data']['n_folds']
    )
    print('KLSData size:', len(dataset))
    if cfg['general']['use_wandb']:
        wandb.log({'kls_data_size': len(dataset)})
else:
    # bowel, extravasation dataset
    print('apply_aug:', cfg['data']['apply_aug'])
    b_e_pos = pd.read_csv(B_E_POS_PATH)
    sample_b_e_neg = pd.read_csv(B_E_NEG_PATH)
    dataset = AbdominalBEData(
        train_df,
        BASE_PATH, # Caution! DATA_PATH if on ec2
        b_e_pos,
        sample_b_e_neg,
        apply_aug=cfg['data']['apply_aug'],
        num_fold=cfg['data']['n_folds']
    )
    print('BEData size:', len(dataset))
    if cfg['general']['use_wandb']:
        wandb.log({'b_e_data_size': len(dataset)})


STRIDE: 1000

N_FOLDS: 5


  0%|          | 0/193065 [00:00<?, ?it/s]

  0%|          | 0/188213 [00:00<?, ?it/s]

  0%|          | 0/198102 [00:00<?, ?it/s]

  0%|          | 0/177646 [00:00<?, ?it/s]

...

  0%|          | 0/186309 [00:00<?, ?it/s]

  0%|          | 0/192002 [00:00<?, ?it/s]

  0%|          | 0/178152 [00:00<?, ?it/s]

Dataset size: 21616


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.

  warnings.warn(


In [ ]:
# provided by competition organizer
import numpy as np
import pandas as pd
import pandas.api.types
import sklearn.metrics


class ParticipantVisibleError(Exception):
    pass


def normalize_probabilities_to_one(df: pd.DataFrame, group_columns: list) -> pd.DataFrame:
    # Normalize the sum of each row's probabilities to 100%.
    # 0.75, 0.75 => 0.5, 0.5
    # 0.1, 0.1 => 0.5, 0.5
    row_totals = df[group_columns].sum(axis=1)
    if row_totals.min() == 0:
        raise ParticipantVisibleError('All rows must contain at least one non-zero prediction')
    for col in group_columns:
        df[col] /= row_totals
    return df


def calc_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str = '') -> float:
    '''
    Pseudocode:
    1. For every label group (liver, bowel, etc):
        - Normalize the sum of each row's probabilities to 100%.
        - Calculate the sample weighted log loss.
    2. Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    3. Calculate the sample weighted log loss for the new label group
    4. Return the average of all of the label group log losses as the final score.
    '''
#     del solution[row_id_column_name]
#     del submission[row_id_column_name]

    # Run basic QC checks on the inputs
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('All submission values must be numeric')

    if not np.isfinite(submission.values).all():
        raise ParticipantVisibleError('All submission values must be finite')

    if solution.min().min() < 0:
        raise ParticipantVisibleError('All labels must be at least zero')
    if submission.min().min() < 0:
        raise ParticipantVisibleError('All predictions must be at least zero')

    # Calculate the label group log losses
    binary_targets = ['bowel', 'extravasation']
    triple_level_targets = ['kidney', 'liver', 'spleen']
    all_target_categories = binary_targets + triple_level_targets

    label_group_losses = []
    for category in all_target_categories:
        if category in binary_targets:
            col_group = [f'{category}_healthy', f'{category}_injury']
        else:
            col_group = [f'{category}_healthy', f'{category}_low', f'{category}_high']

        solution = normalize_probabilities_to_one(solution, col_group)

        for col in col_group:
            if col not in submission.columns:
                raise ParticipantVisibleError(f'Missing submission column {col}')
        submission = normalize_probabilities_to_one(submission, col_group)
        label_group_losses.append(
            sklearn.metrics.log_loss(
                y_true=solution[col_group].values,
                y_pred=submission[col_group].values,
                sample_weight=solution[f'{category}_weight'].values
            )
        )

    # Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    healthy_cols = [x + '_healthy' for x in all_target_categories]
    any_injury_labels = (1 - solution[healthy_cols]).max(axis=1)
    any_injury_predictions = (1 - submission[healthy_cols]).max(axis=1)
    any_injury_loss = sklearn.metrics.log_loss(
        y_true=any_injury_labels.values,
        y_pred=any_injury_predictions.values,
        sample_weight=solution['any_injury_weight'].values
    )

    label_group_losses.append(any_injury_loss)
    return np.mean(label_group_losses)



In [ ]:
def predict(model, test_dataloader):
    model.eval()
    test_predictions = []
    with torch.no_grad():
        for batch_idx, batch_data in tqdm(enumerate(test_dataloader)):
            image = batch_data['image']
            patient_id = batch_data['patient_id']
            b, e, k, l, s = model(image)

            # Apply activations to get probabilities
            b_probs, e_probs, k_probs, l_probs, s_probs = map(
                lambda x: F.softmax(x, dim=1),
                [b, e, k, l, s]
            )

            # Transfer probabilities back to CPU
            b_probs, e_probs, k_probs, l_probs, s_probs = map(
                lambda x: x.cpu().numpy().astype(np.float64),
                [b_probs, e_probs, k_probs, l_probs, s_probs]
            )

            # Get one prediction per series in the batch
            for i in range(b_probs.shape[0]):  # Assuming all arrays have the same size
                test_predictions.append([
                    int(patient_id[i]),
                    *b_probs[i],
                    *e_probs[i],
                    *k_probs[i],
                    *l_probs[i],
                    *s_probs[i]
                ])

    column_names = ['patient_id',
                    'bowel_healthy', 'bowel_injury',
                    'extravasation_healthy', 'extravasation_injury',
                    'kidney_healthy', 'kidney_low', 'kidney_high',
                    'liver_healthy', 'liver_low', 'liver_high',
                    'spleen_healthy', 'spleen_low', 'spleen_high']

    df_preds = pd.DataFrame(test_predictions, columns=column_names)
    df_preds = df_preds.groupby('patient_id').mean().reset_index()

    return df_preds


def calc_cv_score(model, val_dataloader, train_df):
    print("Calculating CV score...")
    preds_df = predict(model, val_dataloader)
    patients = preds_df[['patient_id']]
    solution_df = pd.merge(patients, train_df, on='patient_id', how='left')
    # add weights
    solution_df['bowel_weight'] = 1
    solution_df['extravasation_weight'] = 1
    solution_df['kidney_weight'] = 1
    solution_df['liver_weight'] = 1
    solution_df['spleen_weight'] = 1
    solution_df['any_injury_weight'] = 1

    score = calc_score(solution_df, preds_df)
    print('CV score:', score)
    if cfg['general']['use_wandb']:
        wandb.log({'cv_score': score})


In [11]:
# train and validate
print(f"Training {cfg['model']['model_type']}...")
print('batch_size:', cfg['data']['batch_size'])
print('max_epochs:', cfg['pl_params']['max_epochs'])

print(f"folds: {cfg['data']['fold_i']}")
for fold_i in cfg['data']['fold_i']:
    print(f'fold {fold_i}')

    if cfg['model']['model_type'] == 'kls':
        model = KLSNet(cfg)
    else:
        model = BENet(cfg)

    train_data, val_data = dataset.get_one_fold(fold_i)
    train_dataloader = DataLoader(
        train_data, batch_size=cfg['data']['batch_size'],
        shuffle=True, num_workers=num_workers, pin_memory=True)
    val_dataloader = DataLoader(
        val_data, batch_size=cfg['data']['batch_size'],
        shuffle=False, num_workers=num_workers, pin_memory=True)

    # train for some epochs
    trainer = Trainer(
        logger=logger,
        **cfg['pl_params'], 
    )
    trainer.fit(
        model,
        train_dataloader,
        val_dataloader,
    )

    # calc cv score
    if cfg['data']['calc_cv_score']:
        calc_cv_score(model, val_dataloader, train_df)

    if cfg['general']['use_wandb']:
        wandb.finish()

    # save model
    torch.save(model, f"{MODEL_PATH}_fold{fold_i}.pt")
    del model
    torch.cuda.empty_cache()


BATCH_SIZE: 16

NUM_EPOCHS: 2


Fold 0
